# Imports

In [75]:
import os
from math import isnan
import numpy as np
import pandas as pd

# from ds_aws_services.athena_api  import cachedAthenaApi
from ds_aws_services import CachedAthenaApi
import pandas as pd
# from datetime import datetime, timedelta
import time

os.environ['disk_caching'] = 'true'

# Analysis

### Params:

In [102]:
# Biz2credit BI
partner_ids = [13589]
process_names = ["bi_biz2credit_lead"]
transaction_month_prt = "2025-01"
vertical_ids = ["64e33e7be3cbc4ce1041a30f"]
start_date = "2025-08-01"
end_date = "2025-09-06"
transaction_month_prt_start = "2025-08"
transaction_month_prt_end = "2025-10"

enrichment_cols = "age_of_business_months, application_annual_revenue, business_legal_structure, loan_purpose, industry, sub_industry, users_prob_sale, "
max_enrichment_cols = "MAX(age_of_business_months) AS age_of_business_months, MAX(application_annual_revenue) AS application_annual_revenue, max(business_legal_structure) as business_legal_structure, max(loan_purpose) as loan_purpose, max(industry) as industry, max(sub_industry) as sub_industry, max(users_prob_sale) as users_prob_sale "

In [103]:
# Rocket 
partner_ids = [3158,3178]
process_names = ["quicken_rocket_allocated"]
transaction_month_prt = "2025-01"
vertical_ids = ["5fa2b415c91a2010c3432900"]
start_date = "2025-08-01"
end_date = "2025-09-07"
transaction_month_prt_start = "2025-08"
transaction_month_prt_end = "2025-10"

enrichment_cols = "loanamount, creditscore, "
max_enrichment_cols = "MAX(f.loanamount)  AS loan_amount, MAX(f.creditscore) AS credit_score "

### Query 1 :  Checking RN

In [94]:
import pandas as pd

def get_data1(
    partner_ids,
    process_names,              # e.g. ['quicken_rocket_allocated','quicken_rocket_leads']
    transaction_month_prt,      # e.g. '2025-02' (or None to derive from start_date)
    vertical_ids,
    start_date,                 # e.g. '2025-08-01'
    end_date,                   # not used in this query, kept for params symmetry
    cond1_col=None,             # e.g. 'loanpurpose'
    cond1_val=None              # e.g. 'Refinance' or '%%' to skip
) -> pd.DataFrame:
    # --- Build WHERE for the CTE ---
    query = f"""
WITH enrichment_data AS (
    SELECT  f.subid,
            process_name,
            f.partner_name,
            f.rn,
            transaction_month_prt,
            company,
            MIN(f.rn) OVER (PARTITION BY subid) AS min_rn,
            {max_enrichment_cols}

    FROM dlk_visitor_funnel_dwh_production.enrich_conversions_flatten f
    WHERE f.partner_id in ({','.join(map(str, partner_ids))})
    AND process_name in ({','.join(f"'{p}'" for p in process_names)})
     AND transaction_month_prt >= '{transaction_month_prt}'
     AND f.vertical_id in ({','.join(f"'{v}'" for v in vertical_ids)})

    GROUP BY 1,2,3,4,5,6
)
SELECT
       rn,
       COUNT(DISTINCT subid) AS cids,
       COUNT(subid)          AS rowss
FROM enrichment_data
WHERE rn = min_rn
GROUP BY 1
ORDER BY 2 DESC
"""
    return pd.DataFrame(CachedAthenaApi().execute_fetch(query))

df1 = get_data1(
    partner_ids=partner_ids,
    process_names=process_names,
    transaction_month_prt=transaction_month_prt,
    vertical_ids=vertical_ids,
    start_date=start_date,
    end_date=end_date
)
df1.head()

2025-09-08 16:06:36,624 INFO [ds_logger.py:68] [Cached AthenaApi] Local disk-caching is ENABLED.
2025-09-08 16:06:36,625 INFO [ds_logger.py:68] [Cached AthenaApi] Cache miss for execute_fetch(args=("\nWITH enrichment_data AS (\n    SELECT  f.subid,\n            process_name,\n            f.partner_name,\n            f.rn,\n            transaction_month_prt,\n            company,\n            MIN(f.rn) OVER (PARTITION BY subid) AS min_rn,\n            MAX(f.loanamount)  AS loan_amount, MAX(f.creditscore) AS credit_score \n\n    FROM dlk_visitor_funnel_dwh_production.enrich_conversions_flatten f\n    WHERE f.partner_id in (3158,3178)\n    AND process_name in ('quicken_rocket_allocated')\n     AND transaction_month_prt >= '2025-01'\n     AND f.vertical_id in ('5fa2b415c91a2010c3432900')\n\n    GROUP BY 1,2,3,4,5,6\n)\nSELECT\n       rn,\n       COUNT(DISTINCT subid) AS cids,\n       COUNT(subid)          AS rowss\nFROM enrichment_data\nWHERE rn = min_rn\nGROUP BY 1\nORDER BY 2 DESC\n",), 

,rn,cids,rowss
0,1,82127,82136


### Query 2: checking dates Last Updates (Enrichment Action Time)

In [95]:
def get_data2(
    partner_ids,
    process_names,
    transaction_month_prt,
    vertical_ids,
    start_date,
    end_date
) -> pd.DataFrame:

    query = f"""
SELECT
  CAST(SUBSTRING(action_time, 1, 10) AS DATE) AS action_day,
  COUNT(DISTINCT subid) AS subids
FROM DLK_VISITOR_FUNNEL_DWH_PRODUCTION.ENRICH_CONVERSIONS_FLATTEN
WHERE partner_id in ({','.join(map(str, partner_ids))})
AND process_name in ({','.join(f"'{p}'" for p in process_names)})
 AND transaction_month_prt >= '{transaction_month_prt}'
 AND vertical_id in ({','.join(f"'{v}'" for v in vertical_ids)})
GROUP BY 1
ORDER BY 1 DESC
"""
    return pd.DataFrame(CachedAthenaApi().execute_fetch(query))

df2 = get_data2(
    partner_ids=partner_ids,
    process_names=process_names,
    transaction_month_prt=transaction_month_prt,
    vertical_ids=vertical_ids,
    start_date=start_date,
    end_date=end_date
)
df2.head()    

2025-09-08 16:06:45,585 INFO [ds_logger.py:68] [Cached AthenaApi] Local disk-caching is ENABLED.
2025-09-08 16:06:45,586 INFO [ds_logger.py:68] [Cached AthenaApi] Using cached results for execute_fetch(args=("\nSELECT\n  CAST(SUBSTRING(action_time, 1, 10) AS DATE) AS action_day,\n  COUNT(DISTINCT subid) AS subids\nFROM DLK_VISITOR_FUNNEL_DWH_PRODUCTION.ENRICH_CONVERSIONS_FLATTEN\nWHERE partner_id in (3158,3178)\nAND process_name in ('quicken_rocket_allocated')\n AND transaction_month_prt >= '2025-01'\n AND vertical_id in ('5fa2b415c91a2010c3432900')\nGROUP BY 1\nORDER BY 1 DESC\n",), kwargs={}).


,action_day,subids
0,2025-09-07,1671
1,2025-09-01,9532
2,2025-08-26,1
3,2025-08-14,1
4,2025-08-01,7732


Most subids are updated in the last day! The ones that has previous rows data is because the values changed so they have bigger RN

### Query 3: Per creation date counts

In [96]:

def get_data3(
    partner_ids,
    process_names,
    transaction_month_prt,
    vertical_ids,
    start_date,
    end_date
) -> pd.DataFrame:

    query = f"""
SELECT
  CAST(SUBSTRING(created_at, 1, 10) AS DATE) AS created_at_day,
  COUNT(DISTINCT subid) AS subids
FROM DLK_VISITOR_FUNNEL_DWH_PRODUCTION.ENRICH_CONVERSIONS_FLATTEN
WHERE partner_id in ({','.join(map(str, partner_ids))})
AND process_name in ({','.join(f"'{p}'" for p in process_names)})
 AND transaction_month_prt >= '{transaction_month_prt}'
 AND vertical_id in ({','.join(f"'{v}'" for v in vertical_ids)})
 -- and rn = 1
GROUP BY 1
ORDER BY 1 DESC
"""
    return pd.DataFrame(CachedAthenaApi().execute_fetch(query))

df3 = get_data3(
    partner_ids=partner_ids,
    process_names=process_names,
    transaction_month_prt=transaction_month_prt,
    vertical_ids=vertical_ids,
    start_date=start_date,
    end_date=end_date
)
df3.head()        

2025-09-08 16:06:45,604 INFO [ds_logger.py:68] [Cached AthenaApi] Local disk-caching is ENABLED.
2025-09-08 16:06:45,608 INFO [ds_logger.py:68] [Cached AthenaApi] Cache miss for execute_fetch(args=("\nSELECT\n  CAST(SUBSTRING(created_at, 1, 10) AS DATE) AS created_at_day,\n  COUNT(DISTINCT subid) AS subids\nFROM DLK_VISITOR_FUNNEL_DWH_PRODUCTION.ENRICH_CONVERSIONS_FLATTEN\nWHERE partner_id in (3158,3178)\nAND process_name in ('quicken_rocket_allocated')\n AND transaction_month_prt >= '2025-01'\n AND vertical_id in ('5fa2b415c91a2010c3432900')\n -- and rn = 1\nGROUP BY 1\nORDER BY 1 DESC\n",), kwargs={}).
fetching manifest from s3://aws-athena-query-results-925511037392-us-east-1/Unsaved/2025/09/08/dee9be1d-6212-4540-8c80-4a63ccc3b7dc-manifest.csv
2025-09-08 16:06:50,493 INFO [ds_logger.py:68] the function _execute_unload was executed in 4.87444 seconds
2025-09-08 16:06:50,657 INFO [ds_logger.py:68] the function _execute_fetch was executed in 5.03782 seconds


,created_at_day,subids
0,2025-09-07,210
1,2025-09-06,320
2,2025-09-05,313
3,2025-09-04,287
4,2025-09-03,500


Looks like every day added around 60 +- subids.  looks valid

### Query 4: Hours diff for each field

In [97]:
import pandas as pd
import numpy as np

def get_data3_combined(
    partner_ids,
    process_names,
    transaction_month_prt,  # not used (we derive months from dates; kept for symmetry)
    vertical_ids,
    start_date,  # 'YYYY-MM-DD'
    end_date     # 'YYYY-MM-DD'
) -> pd.DataFrame:
    tm_start = start_date[:7]
    tm_end   = end_date[:7]

    query = f"""
WITH raw_enrich AS (
  SELECT DISTINCT
      subid,
      process_name,
      CAST(SUBSTRING(transaction_date, 1, 10) AS DATE) AS transaction_day,
      transaction_date,
      transaction_month_prt,
      created_at,
      {enrichment_cols}
      rn
  FROM DLK_VISITOR_FUNNEL_DWH_PRODUCTION.ENRICH_CONVERSIONS_FLATTEN
  WHERE partner_id in ({','.join(map(str, partner_ids))})
    AND process_name in ({','.join(f"'{p}'" for p in process_names)})
    AND transaction_month_prt between '{tm_start}' and '{tm_end}'
    AND vertical_id in ({','.join(f"'{v}'" for v in vertical_ids)})
    AND loanpurpose = 'Refinance'
    AND rn = 1
),
ff AS (
  SELECT
      cid,
      conversion_date,
      conversion_timestamp,
      clickout_timestamp,
      company,
      SUM(leads_count) AS leads,
      SUM(qualified_leads_count) AS qls,
      SUM(sales_count) AS sales
  FROM dlk_visitor_funnel_dwh_production.chart_funnel
  WHERE partner_id in ({','.join(map(str, partner_ids))})
    AND dt between '{tm_start}' and '{tm_end}'
    AND vertical_id in ({','.join(f"'{v}'" for v in vertical_ids)})
    AND traffic_type = 'users'
  GROUP BY 1,2,3,4,5
  HAVING SUM(leads_count) >= 1
),
combined AS (
  SELECT
      r.subid,
      ff.company,
      r.transaction_date,
      date_format(date_trunc('millisecond', MAX(ff.clickout_timestamp)),
        '%Y-%m-%d %H:%i:%s.%f') AS clickout_ts_mt,

      date_format( date_trunc( 'millisecond', MIN(CASE WHEN r.creditscore IS NOT NULL THEN
              COALESCE(
                date_parse(SUBSTRING(r.created_at, 1, 19), '%Y-%m-%dT%H:%i:%s'),
                from_iso8601_timestamp(regexp_replace(r.created_at, '([+-]\\d{2})(\\d{2})$', '\\1:\\2'))
              )
            END
          )
        ),
        '%Y-%m-%d %H:%i:%s.%f'
      ) AS creditscore_time,

      /* loanamount_time → only when present; parse flexibly, ms-trunc, stringify */
      date_format(
        date_trunc(
          'millisecond',
          MIN(
            CASE WHEN r.loanamount IS NOT NULL THEN
              COALESCE(
                date_parse(SUBSTRING(r.created_at, 1, 19), '%Y-%m-%dT%H:%i:%s'),
                date_parse(SUBSTRING(r.created_at, 1, 19), '%Y-%m-%d %H:%i:%s'),
                from_iso8601_timestamp(regexp_replace(r.created_at, '([+-]\\d{2})(\\d{2})$', '\\1:\\2'))
              )
            END
          )
        ),
        '%Y-%m-%d %H:%i:%s.%f'
      ) AS loanamount_time,
      sum(ff.leads) as leads,
      sum(ff.qls) as qls,
      sum(ff.sales) as sales
      
  FROM raw_enrich r
  INNER JOIN ff
    ON r.subid = ff.cid
   AND r.transaction_day >= ff.conversion_date
  GROUP BY r.subid, r.transaction_date, ff.company
)
SELECT * FROM combined
"""
    return pd.DataFrame(CachedAthenaApi().execute_fetch(query))

combined_df = get_data3_combined(
    partner_ids=partner_ids,
    process_names=process_names,
    transaction_month_prt='2025-08',   # ignored for month range; kept for signature symmetry
    vertical_ids=vertical_ids,
    start_date='2025-08-01',
    end_date='2025-09-06'
)

# parse the string timestamps before hours math (your build_hours_summary already does this)
for col in ["clickout_ts_mt", "creditscore_time", "loanamount_time", "transaction_date"]:
    combined_df[col] = pd.to_datetime(combined_df[col], errors="coerce", utc=True)

# Add hour difference columns for all columns ending with '_time'
def add_hour_diffs(df, reference_col='clickout_ts_mt'):
    """
    Add hour difference columns for all columns ending with '_time'
    compared to the reference column (default: clickout_ts_mt)
    """
    df = df.copy()
    
    # Find all columns ending with '_time'
    time_cols = [col for col in df.columns if col.endswith('_time')]
    
    for time_col in time_cols:
        # Create diff column name
        diff_col = time_col.replace('_time', '_diff_hours')
        
        # Calculate hour difference: (time_col - reference_col) in hours
        # Floor the result and ensure non-negative values
        time_diff = (df[time_col] - df[reference_col]).dt.total_seconds() / 3600.0
        time_diff = np.floor(time_diff)  # Floor like TIMESTAMPDIFF('hour',...)
        time_diff = np.maximum(time_diff, 0)  # Ensure non-negative (GREATEST(..., 0))
        
        # Handle NaN values (when either timestamp is null)
        time_diff = np.where(df[time_col].isna() | df[reference_col].isna(), np.nan, time_diff)
        
        df[diff_col] = time_diff
    
    return df

# Apply the function to add hour difference columns
combined_df = add_hour_diffs(combined_df)

print("Added hour difference columns:")
print("Columns:", combined_df.columns.tolist())
print("\nFirst 5 rows with hour differences:")
combined_df.head()



2025-09-08 16:06:50,671 INFO [ds_logger.py:68] [Cached AthenaApi] Local disk-caching is ENABLED.
2025-09-08 16:06:50,672 INFO [ds_logger.py:68] [Cached AthenaApi] Using cached results for execute_fetch(args=("\nWITH raw_enrich AS (\n  SELECT DISTINCT\n      subid,\n      process_name,\n      CAST(SUBSTRING(transaction_date, 1, 10) AS DATE) AS transaction_day,\n      transaction_date,\n      transaction_month_prt,\n      created_at,\n      loanamount, creditscore, \n      rn\n  FROM DLK_VISITOR_FUNNEL_DWH_PRODUCTION.ENRICH_CONVERSIONS_FLATTEN\n  WHERE partner_id in (3158,3178)\n    AND process_name in ('quicken_rocket_allocated')\n    AND transaction_month_prt between '2025-08' and '2025-09'\n    AND vertical_id in ('5fa2b415c91a2010c3432900')\n    AND loanpurpose = 'Refinance'\n    AND rn = 1\n),\nff AS (\n  SELECT\n      cid,\n      conversion_date,\n      conversion_timestamp,\n      clickout_timestamp,\n      company,\n      SUM(leads_count) AS leads,\n      SUM(qualified_leads_coun

,subid,company,transaction_date,clickout_ts_mt,creditscore_time,loanamount_time,leads,qls,sales,creditscore_diff_hours,loanamount_diff_hours
0,xiISNkaje1,ni,2025-08-30 21:00:00+00:00,2025-08-31 16:26:37+00:00,2025-09-01 18:02:55+00:00,2025-09-01 18:02:55+00:00,1,0,0,25.0,25.0
1,NaK5O5DG1c,ni,2025-08-20 21:00:00+00:00,2025-08-21 22:11:50+00:00,2025-08-22 18:05:42+00:00,2025-08-22 18:05:42+00:00,1,0,0,19.0,19.0
2,Xmeu7xg3TB,ni,2025-08-15 21:00:00+00:00,2025-08-16 19:53:32+00:00,2025-08-17 18:02:07+00:00,2025-08-17 18:02:07+00:00,1,0,0,22.0,22.0
3,lms4pwxLh7,ni,2025-09-03 21:00:00+00:00,2025-08-30 09:07:26+00:00,2025-09-05 18:00:51+00:00,2025-09-05 18:00:51+00:00,1,0,0,152.0,152.0
4,Kfas0v0f2f,ni,2025-08-19 21:00:00+00:00,2025-08-20 12:43:07+00:00,2025-08-21 18:01:32+00:00,2025-08-21 18:01:32+00:00,1,0,0,29.0,29.0


### Query 5: aggregated hours diff

In [98]:
# Fixed version with sales stats included
def compute_field_stats_fixed(df: pd.DataFrame) -> pd.DataFrame:
    # Find all columns ending with '_diff_hours' or '_hours'
    hour_cols = [col for col in df.columns if col.endswith('_diff_hours') or col.endswith('_hours')]
    
    if not hour_cols:
        print("No hour columns found. Available columns:", df.columns.tolist())
        return pd.DataFrame()
    
    print(f"Using hour columns: {hour_cols}")
    
    # Check if sales column exists
    has_sales = 'sales' in df.columns
    print(f"Sales column available: {has_sales}")
    
    long_df = df.melt(
        id_vars=["subid", "company", "transaction_date", "clickout_ts_mt"] + (["sales"] if has_sales else []),
        value_vars=hour_cols,
        var_name="field_name",
        value_name="hours"
    )
    long_df["field_name"] = long_df["field_name"].str.upper()

    g = long_df.groupby(["company", "field_name"], dropna=False)
    avg_hours     = g["hours"].mean()
    median_hours  = g["hours"].quantile(0.50, interpolation="linear")
    p80_hours     = g["hours"].quantile(0.80, interpolation="linear")
    p90_hours     = g["hours"].quantile(0.90, interpolation="linear")
    filled_rows   = g["hours"].count()
    total_rows    = g.size()

    users_with_value = long_df[~long_df["hours"].isna()].groupby(["company","field_name"])["subid"].nunique()
    total_users      = long_df.groupby(["company","field_name"])["subid"].nunique()

    # Calculate sales stats if sales column exists
    if has_sales:
        avg_sales = g["sales"].mean()
        sum_sales = g["sales"].sum()
        
        out = pd.concat(
            [avg_hours.rename("avg_hours"),
             median_hours.rename("median_hours"),
             p80_hours.rename("p80_hours"),
             p90_hours.rename("p90_hours"),
             filled_rows.rename("filled_rows"),
             total_rows.rename("total_rows"),
             users_with_value.rename("users_with_value"),
             total_users.rename("total_users"),
             avg_sales.rename("avg_sales"),
             sum_sales.rename("sum_sales")],
            axis=1
        ).reset_index()
        
        out["null_rate_rows"]  = (1 - (out["filled_rows"] / out["total_rows"].replace(0, np.nan))).round(4)
        out["null_rate_users"] = (1 - (out["users_with_value"] / out["total_users"].replace(0, np.nan))).round(4)

        out = out[["company","field_name","avg_hours","median_hours","p80_hours","p90_hours",
                   "filled_rows","total_rows","null_rate_rows","users_with_value","total_users","null_rate_users",
                   "avg_sales","sum_sales"]]\
                .sort_values(["company","field_name"]).reset_index(drop=True)
    else:
        out = pd.concat(
            [avg_hours.rename("avg_hours"),
             median_hours.rename("median_hours"),
             p80_hours.rename("p80_hours"),
             p90_hours.rename("p90_hours"),
             filled_rows.rename("filled_rows"),
             total_rows.rename("total_rows"),
             users_with_value.rename("users_with_value"),
             total_users.rename("total_users")],
            axis=1
        ).reset_index()
        
        out["null_rate_rows"]  = (1 - (out["filled_rows"] / out["total_rows"].replace(0, np.nan))).round(4)
        out["null_rate_users"] = (1 - (out["users_with_value"] / out["total_users"].replace(0, np.nan))).round(4)

        out = out[["company","field_name","avg_hours","median_hours","p80_hours","p90_hours",
                   "filled_rows","total_rows","null_rate_rows","users_with_value","total_users","null_rate_users"]]\
                .sort_values(["company","field_name"]).reset_index(drop=True)
    
    return out

# Use the fixed function
field_stats = compute_field_stats_fixed(combined_df)    

# Print null rates per field
print("\n" + "="*60)
print("NULL RATES PER FIELD:")
print("="*60)
for _, row in field_stats.iterrows():
    print(f"Field: {row['field_name']}")
    print(f"  Row null rate: {row['null_rate_rows']:.4f} ({row['null_rate_rows']*100:.2f}%)")
    print(f"  User null rate: {row['null_rate_users']:.4f} ({row['null_rate_users']*100:.2f}%)")
    print(f"  Filled rows: {row['filled_rows']} / {row['total_rows']}")
    print(f"  Users with value: {row['users_with_value']} / {row['total_users']}")
    if 'avg_sales' in row:
        print(f"  Average sales: {row['avg_sales']:.2f}")
        print(f"  Total sales: {row['sum_sales']:.0f}")
    print("-" * 40)

print("\n" + "="*60)
print("SUMMARY STATISTICS:")
print("="*60)
field_stats.head(20)


Using hour columns: ['creditscore_diff_hours', 'loanamount_diff_hours']
Sales column available: True

NULL RATES PER FIELD:
Field: CREDITSCORE_DIFF_HOURS
  Row null rate: 0.0000 (0.00%)
  User null rate: 0.0000 (0.00%)
  Filled rows: 4663 / 4663
  Users with value: 4663 / 4663
  Average sales: 0.02
  Total sales: 106
----------------------------------------
Field: LOANAMOUNT_DIFF_HOURS
  Row null rate: 0.0000 (0.00%)
  User null rate: 0.0000 (0.00%)
  Filled rows: 4663 / 4663
  Users with value: 4663 / 4663
  Average sales: 0.02
  Total sales: 106
----------------------------------------

SUMMARY STATISTICS:


,company,field_name,avg_hours,median_hours,p80_hours,p90_hours,filled_rows,total_rows,null_rate_rows,users_with_value,total_users,null_rate_users,avg_sales,sum_sales
0,ni,CREDITSCORE_DIFF_HOURS,31.364787,25.0,31.0,46.0,4663,4663,0.0,4663,4663,0.0,0.022732,106
1,ni,LOANAMOUNT_DIFF_HOURS,31.364787,25.0,31.0,46.0,4663,4663,0.0,4663,4663,0.0,0.022732,106


### Query 6: for general data for analysis:

In [116]:
def get_general_data(
    partner_ids,
    process_names,
    transaction_month_prt,  # not used (we derive months from dates; kept for symmetry)
    vertical_ids,
    start_date,  # 'YYYY-MM-DD'
    end_date     # 'YYYY-MM-DD'
) -> pd.DataFrame:
    tm_start = start_date[:7]
    tm_end   = end_date[:7]

    query = f"""
WITH raw_enrich AS (
  SELECT DISTINCT
      subid,
      process_name,
      transaction_date,
      CAST(SUBSTRING(transaction_date, 1, 10) AS DATE) AS transaction_day,
      transaction_month_prt,
      {enrichment_cols}
      row_number () over (partition by subid order by transaction_date desc) as rnn

  FROM DLK_VISITOR_FUNNEL_DWH_PRODUCTION.ENRICH_CONVERSIONS_FLATTEN
  WHERE partner_id in ({','.join(map(str, partner_ids))})
    AND process_name in ({','.join(f"'{p}'" for p in process_names)})
    AND transaction_month_prt between '{tm_start}' and '{tm_end}'
    AND vertical_id in ({','.join(f"'{v}'" for v in vertical_ids)})
    AND rn = 1 
    AND loanpurpose = 'Refinance'
),
ff AS (
  SELECT
      cid,
      conversion_date,
      conversion_timestamp,
      company,
      SUM(leads_count) AS leads,
      SUM(qualified_leads_count) AS qls,
      SUM(sales_count) AS sales

  FROM dlk_visitor_funnel_dwh_production.chart_funnel
  WHERE partner_id in ({','.join(map(str, partner_ids))})
    AND dt between '{tm_start}' and '{tm_end}'
    AND vertical_id in ({','.join(f"'{v}'" for v in vertical_ids)})
    AND traffic_type = 'users'
  GROUP BY 1,2,3,4
  HAVING SUM(leads_count) >= 1
),
combined AS (
  SELECT
      coalesce(ff.cid, r.subid) as subid,
      ff.company,
      r.transaction_date,
      {enrichment_cols}
      sum(ff.leads) as leads,
      sum(ff.qls) as qls,
      sum(ff.sales) as sales
      
  FROM ff 
  LEFT JOIN raw_enrich r
    ON r.subid = ff.cid
   AND r.transaction_day >= ff.conversion_date and rnn = 1 
  GROUP BY 1,2,3,4,5
)
SELECT * FROM combined """

    return pd.DataFrame(CachedAthenaApi().execute_fetch(query))

analysis_df = get_general_data(
    partner_ids=partner_ids,
    process_names=process_names,
    transaction_month_prt='2025-08',   # ignored for month range; kept for signature symmetry
    vertical_ids=vertical_ids,
    start_date='2025-08-01',
    end_date='2025-09-06'
)

analysis_df.head

2025-09-08 16:20:22,015 INFO [ds_logger.py:68] [Cached AthenaApi] Local disk-caching is ENABLED.
2025-09-08 16:20:22,016 INFO [ds_logger.py:68] [Cached AthenaApi] Cache miss for execute_fetch(args=("\nWITH raw_enrich AS (\n  SELECT DISTINCT\n      subid,\n      process_name,\n      transaction_date,\n      CAST(SUBSTRING(transaction_date, 1, 10) AS DATE) AS transaction_day,\n      transaction_month_prt,\n      loanamount, creditscore, \n      row_number () over (partition by subid order by transaction_date desc) as rnn\n\n  FROM DLK_VISITOR_FUNNEL_DWH_PRODUCTION.ENRICH_CONVERSIONS_FLATTEN\n  WHERE partner_id in (3158,3178)\n    AND process_name in ('quicken_rocket_allocated')\n    AND transaction_month_prt between '2025-08' and '2025-09'\n    AND vertical_id in ('5fa2b415c91a2010c3432900')\n    AND rn = 1 \n    AND loanpurpose = 'Refinance'\n),\nff AS (\n  SELECT\n      cid,\n      conversion_date,\n      conversion_timestamp,\n      company,\n      SUM(leads_count) AS leads,\n      SU

<bound method NDFrame.head of             subid company          transaction_date  loanamount  \
0      YhF18RwGVb      ni  2025-08-14T00:00:00+0300    805000.0   
1      buMP1gUMPf      ni  2025-08-18T00:00:00+0300    355000.0   
2      j3RwW11LsG      ni  2025-08-18T00:00:00+0300    118000.0   
3      Mc9wywjc17      ni  2025-08-29T00:00:00+0300    159000.0   
4      suDz2ypGg6      ni  2025-08-11T00:00:00+0300     60000.0   
...           ...     ...                       ...         ...   
22003  xk1ew3gbFc      ni  2025-08-06T00:00:00+0300    202800.0   
22004  fCoKu0gpok      ni  2025-08-01T00:00:00+0300     50000.0   
22005  UTjjyzCsw5      ni  2025-08-27T00:00:00+0300    127000.0   
22006  uu7ZYgR7wq      ni  2025-08-12T00:00:00+0300    311000.0   
22007  lshw0JvTcA      ni  2025-08-17T00:00:00+0300     60000.0   

         creditscore  leads  qls  sales  
0               Good      1    0      0  
1          Excellent      1    0      0  
2               Good      1    0      0

In [131]:
analysis_df.sort_values(by='leads', ascending=False).head(10)

,subid,company,transaction_date,loanamount,creditscore,leads,qls,sales,l2s
17406,H6tVMjzhF7,ni,2025-09-03T00:00:00+0300,240000.0,Good,2,0,0,0.0
16081,5dBAiZv3XM,ni,None,NaN,None,2,0,0,0.0
0,YhF18RwGVb,ni,2025-08-14T00:00:00+0300,805000.0,Good,1,0,0,0.0
14668,OEeJQh0xjc,ni,2025-08-04T00:00:00+0300,800000.0,Good,1,0,0,0.0
14676,0euLnXu3dG,ni,None,NaN,None,1,0,0,0.0
14675,IUYgxjjq2n,ni,None,NaN,None,1,0,0,0.0
14674,hC2cFTJc7Y,ni,None,NaN,None,1,0,0,0.0
14673,QhRa1eP3Sg,ni,2025-08-04T00:00:00+0300,450000.0,Average,1,0,0,0.0
14672,dKYiSGM8vZ,ni,2025-08-14T00:00:00+0300,10000.0,Average,1,0,0,0.0
14671,a7EL6jDrpd,ni,None,NaN,None,1,0,0,0.0


#### Additional Analysis

In [132]:
# 1. NULL RATES FOR ENRICHMENT FIELDS
print("="*60)
print("1. NULL RATES FOR ENRICHMENT FIELDS")
print("="*60)

# Identify enrichment fields (exclude subid, company, transaction_date, leads, qls, sales)
enrichment_fields = [col for col in analysis_df.columns if col not in ['subid', 'company', 'transaction_date', 'leads', 'qls', 'sales']]
print(f"Enrichment fields found: {enrichment_fields}")

null_rates = []
for field in enrichment_fields:
    total_rows = len(analysis_df)
    null_rows = analysis_df[field].isnull().sum()
    null_rate = (null_rows / total_rows) * 100
    
    null_rates.append({
        'field': field,
        'total_rows': total_rows,
        'null_rows': null_rows,
        'null_rate_pct': null_rate,
        'filled_rows': total_rows - null_rows
    })
    
    print(f"{field}:")
    print(f"  Total rows: {total_rows}")
    print(f"  Null rows: {null_rows}")
    print(f"  Filled rows: {total_rows - null_rows}")
    print(f"  Null rate: {null_rate:.2f}%")
    print("-" * 40)

# Create summary DataFrame
null_summary = pd.DataFrame(null_rates)
print("\nSUMMARY TABLE:")
print(null_summary[['field', 'null_rate_pct', 'filled_rows', 'total_rows']].round(2))


1. NULL RATES FOR ENRICHMENT FIELDS
Enrichment fields found: ['loanamount', 'creditscore', 'l2s']
loanamount:
  Total rows: 22008
  Null rows: 17345
  Filled rows: 4663
  Null rate: 78.81%
----------------------------------------
creditscore:
  Total rows: 22008
  Null rows: 17345
  Filled rows: 4663
  Null rate: 78.81%
----------------------------------------
l2s:
  Total rows: 22008
  Null rows: 0
  Filled rows: 22008
  Null rate: 0.00%
----------------------------------------

SUMMARY TABLE:
         field  null_rate_pct  filled_rows  total_rows
0   loanamount          78.81         4663       22008
1  creditscore          78.81         4663       22008
2          l2s           0.00        22008       22008


In [133]:
# 2. SALES AND QLS MEAN AND SUMS PER ENRICHMENT FIELD VALUES
print("="*60)
print("2. SALES AND QLS ANALYSIS BY ENRICHMENT FIELD VALUES")
print("="*60)

# Calculate L2S (Leads to Sales) conversion rate
analysis_df['l2s'] = analysis_df['sales'] / analysis_df['leads'].replace(0, np.nan)

for field in enrichment_fields:
    print(f"\n{field.upper()}:")
    print("-" * 40)
    
    # Group by field value and calculate metrics
    field_stats = analysis_df.groupby(field).agg({
        'leads': ['count', 'sum', 'mean'],
        'qls': ['sum', 'mean'],
        'sales': ['sum', 'mean'],
        'l2s': 'mean'
    }).round(4)
    
    # Flatten column names
    field_stats.columns = ['_'.join(col).strip() for col in field_stats.columns]
    field_stats = field_stats.reset_index()
    
    # Rename columns for clarity
    field_stats = field_stats.rename(columns={
        'leads_count': 'records',
        'leads_sum': 'total_leads',
        'qls_sum': 'total_qls',
        'qls_mean': 'avg_qls_per_record',
        'sales_sum': 'total_sales',
        'sales_mean': 'avg_sales_per_record',
        'l2s_mean': 'avg_l2s_rate'
    })
    
    # Sort by total sales descending
    field_stats = field_stats.sort_values('total_sales', ascending=False)
    
    print(f"Top 10 values by total sales:")
    print(field_stats.head(10).to_string(index=False))
    
    # Overall summary for this field
    print(f"\nOverall summary for {field}:")
    print(f"  Total records: {len(analysis_df)}")
    print(f"  Unique values: {analysis_df[field].nunique()}")
    print(f"  Total leads: {analysis_df['leads'].sum()}")
    print(f"  Total QLS: {analysis_df['qls'].sum()}")
    print(f"  Total sales: {analysis_df['sales'].sum()}")
    print(f"  Overall L2S rate: {(analysis_df['sales'].sum() / analysis_df['leads'].sum() * 100):.2f}%")
    print("=" * 60)


2. SALES AND QLS ANALYSIS BY ENRICHMENT FIELD VALUES

LOANAMOUNT:
----------------------------------------
Top 10 values by total sales:
 loanamount  records  total_leads  leads_mean  total_qls  avg_qls_per_record  total_sales  avg_sales_per_record  avg_l2s_rate
    50000.0      112          112         1.0          0                 0.0            7                0.0625        0.0625
   100000.0      113          113         1.0          0                 0.0            3                0.0265        0.0265
   375000.0       18           18         1.0          0                 0.0            3                0.1667        0.1667
   293000.0        5            5         1.0          0                 0.0            3                0.6000        0.6000
   160000.0       21           21         1.0          0                 0.0            3                0.1429        0.1429
   300000.0       63           63         1.0          0                 0.0            2                0.

In [134]:
# 3. PER COMPANY L2S ANALYSIS
print("="*60)
print("3. PER COMPANY L2S ANALYSIS")
print("="*60)

# Company-level analysis
company_stats = analysis_df.groupby('company').agg({
    'subid': 'count',
    'leads': 'sum',
    'qls': 'sum', 
    'sales': 'sum',
    'l2s': 'mean'
}).round(4)

company_stats = company_stats.rename(columns={
    'subid': 'total_records',
    'leads': 'total_leads',
    'qls': 'total_qls',
    'sales': 'total_sales',
    'l2s': 'avg_l2s_rate'
})

# Calculate additional metrics
company_stats['l2s_overall'] = (company_stats['total_sales'] / company_stats['total_leads'] * 100).round(2)
company_stats['qls_rate'] = (company_stats['total_qls'] / company_stats['total_leads'] * 100).round(2)
company_stats['sales_rate'] = (company_stats['total_sales'] / company_stats['total_leads'] * 100).round(2)

# Sort by total sales
company_stats = company_stats.sort_values('total_sales', ascending=False)

print("Company Performance Summary:")
print(company_stats.to_string())

print(f"\nOverall Summary:")
print(f"  Total companies: {len(company_stats)}")
print(f"  Total records: {company_stats['total_records'].sum()}")
print(f"  Total leads: {company_stats['total_leads'].sum()}")
print(f"  Total QLS: {company_stats['total_qls'].sum()}")
print(f"  Total sales: {company_stats['total_sales'].sum()}")
print(f"  Overall L2S rate: {(company_stats['total_sales'].sum() / company_stats['total_leads'].sum() * 100):.2f}%")


3. PER COMPANY L2S ANALYSIS
Company Performance Summary:
         total_records  total_leads  total_qls  total_sales  avg_l2s_rate  l2s_overall  qls_rate  sales_rate
company                                                                                                     
ni               22008        22010        329          107        0.0049         0.49      1.49        0.49

Overall Summary:
  Total companies: 1
  Total records: 22008
  Total leads: 22010
  Total QLS: 329
  Total sales: 107
  Overall L2S rate: 0.49%


In [135]:
# 4. CORRELATION ANALYSIS AND CATEGORICAL ANALYSIS
print("="*60)
print("4. CORRELATION ANALYSIS AND CATEGORICAL ANALYSIS")
print("="*60)

# Function to detect if field is numeric
def is_numeric_field(series):
    try:
        pd.to_numeric(series, errors='raise')
        return True
    except:
        return False

for field in enrichment_fields:
    print(f"\n{field.upper()}:")
    print("-" * 40)
    
    # Check if field is numeric
    is_numeric = is_numeric_field(analysis_df[field])
    print(f"Field type: {'Numeric' if is_numeric else 'Categorical'}")
    
    if is_numeric:
        # Correlation analysis for numeric fields
        print("\nCORRELATION WITH SALES:")
        # Remove null values for correlation
        numeric_data = analysis_df[[field, 'sales', 'leads', 'qls', 'l2s']].dropna()
        
        if len(numeric_data) > 1:
            correlations = {
                'sales': numeric_data[field].corr(numeric_data['sales']),
                'leads': numeric_data[field].corr(numeric_data['leads']),
                'qls': numeric_data[field].corr(numeric_data['qls']),
                'l2s': numeric_data[field].corr(numeric_data['l2s'])
            }
            
            for metric, corr in correlations.items():
                print(f"  {field} vs {metric}: {corr:.4f}")
        else:
            print("  Not enough data for correlation analysis")
            
    else:
        # Categorical analysis
        print("\nTOP 10 CATEGORIES BY SALES:")
        cat_analysis = analysis_df.groupby(field).agg({
            'leads': 'sum',
            'qls': 'sum',
            'sales': 'sum',
            'l2s': 'mean'
        }).round(4)
        
        # Calculate L2S rate
        cat_analysis['l2s_rate'] = (cat_analysis['sales'] / cat_analysis['leads'] * 100).round(2)
        
        # Sort by sales
        cat_analysis = cat_analysis.sort_values('sales', ascending=False)
        
        # Show top 10
        top_10 = cat_analysis.head(10)
        print(f"{'Category':<20} {'Leads':<10} {'QLS':<10} {'Sales':<10} {'L2S Rate':<10}")
        print("-" * 70)
        for category, row in top_10.iterrows():
            print(f"{str(category)[:18]:<20} {row['leads']:<10.0f} {row['qls']:<10.0f} {row['sales']:<10.0f} {row['l2s_rate']:<10.2f}%")
        
        print(f"\nTotal categories: {len(cat_analysis)}")
        print(f"Categories with sales > 0: {len(cat_analysis[cat_analysis['sales'] > 0])}")
    
    print("=" * 60)


4. CORRELATION ANALYSIS AND CATEGORICAL ANALYSIS

LOANAMOUNT:
----------------------------------------
Field type: Numeric

CORRELATION WITH SALES:
  loanamount vs sales: -0.0211
  loanamount vs leads: -0.0024
  loanamount vs qls: nan
  loanamount vs l2s: -0.0211

CREDITSCORE:
----------------------------------------
Field type: Categorical

TOP 10 CATEGORIES BY SALES:
Category             Leads      QLS        Sales      L2S Rate  
----------------------------------------------------------------------
Excellent            2350       0          64         2.72      %
Good                 1098       0          33         3.01      %
Average              641        0          8          1.25      %
Below Average        507        0          1          0.20      %
Poor                 63         0          0          0.00      %
Unknown              5          0          0          0.00      %

Total categories: 6
Categories with sales > 0: 4

L2S:
----------------------------------------

/opt/anaconda3/envs/checking-new-partners-enrichment/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/anaconda3/envs/checking-new-partners-enrichment/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [136]:
# 4. CORRELATION ANALYSIS AND CATEGORICAL ANALYSIS (FIXED VERSION)
print("="*60)
print("4. CORRELATION ANALYSIS AND CATEGORICAL ANALYSIS")
print("="*60)

# Function to detect if field is numeric
def is_numeric_field(series):
    try:
        pd.to_numeric(series, errors='raise')
        return True
    except:
        return False

# Function to safely calculate correlation
def safe_correlation(series1, series2):
    try:
        # Remove null values
        data = pd.DataFrame({'x': series1, 'y': series2}).dropna()
        if len(data) < 2:
            return np.nan
        # Check if either series has zero variance
        if data['x'].std() == 0 or data['y'].std() == 0:
            return np.nan
        return data['x'].corr(data['y'])
    except:
        return np.nan

for field in enrichment_fields:
    print(f"\n{field.upper()}:")
    print("-" * 40)
    
    # Check if field is numeric
    is_numeric = is_numeric_field(analysis_df[field])
    print(f"Field type: {'Numeric' if is_numeric else 'Categorical'}")
    
    if is_numeric:
        # Correlation analysis for numeric fields
        print("\nCORRELATION WITH SALES:")
        # Remove null values for correlation
        numeric_data = analysis_df[[field, 'sales', 'leads', 'qls', 'l2s']].dropna()
        
        if len(numeric_data) > 1:
            correlations = {
                'sales': safe_correlation(numeric_data[field], numeric_data['sales']),
                'leads': safe_correlation(numeric_data[field], numeric_data['leads']),
                'qls': safe_correlation(numeric_data[field], numeric_data['qls']),
                'l2s': safe_correlation(numeric_data[field], numeric_data['l2s'])
            }
            
            for metric, corr in correlations.items():
                if np.isnan(corr):
                    print(f"  {field} vs {metric}: No correlation (constant values or insufficient data)")
                else:
                    print(f"  {field} vs {metric}: {corr:.4f}")
        else:
            print("  Not enough data for correlation analysis")
            
    else:
        # Categorical analysis
        print("\nTOP 10 CATEGORIES BY SALES:")
        cat_analysis = analysis_df.groupby(field).agg({
            'leads': 'sum',
            'qls': 'sum',
            'sales': 'sum',
            'l2s': 'mean'
        }).round(4)
        
        # Calculate L2S rate safely (avoid division by zero)
        cat_analysis['l2s_rate'] = np.where(
            cat_analysis['leads'] > 0,
            (cat_analysis['sales'] / cat_analysis['leads'] * 100).round(2),
            0
        )
        
        # Sort by sales
        cat_analysis = cat_analysis.sort_values('sales', ascending=False)
        
        # Show top 10
        top_10 = cat_analysis.head(10)
        print(f"{'Category':<20} {'Leads':<10} {'QLS':<10} {'Sales':<10} {'L2S Rate':<10}")
        print("-" * 70)
        for category, row in top_10.iterrows():
            print(f"{str(category)[:18]:<20} {row['leads']:<10.0f} {row['qls']:<10.0f} {row['sales']:<10.0f} {row['l2s_rate']:<10.2f}%")
        
        print(f"\nTotal categories: {len(cat_analysis)}")
        print(f"Categories with sales > 0: {len(cat_analysis[cat_analysis['sales'] > 0])}")
        
        # Additional insights for categorical data
        print(f"\nCategory Performance Insights:")
        print(f"  Best L2S rate: {cat_analysis[cat_analysis['leads'] > 0]['l2s_rate'].max():.2f}%")
        print(f"  Worst L2S rate: {cat_analysis[cat_analysis['leads'] > 0]['l2s_rate'].min():.2f}%")
        print(f"  Average L2S rate: {cat_analysis[cat_analysis['leads'] > 0]['l2s_rate'].mean():.2f}%")
    
    print("=" * 60)


4. CORRELATION ANALYSIS AND CATEGORICAL ANALYSIS

LOANAMOUNT:
----------------------------------------
Field type: Numeric

CORRELATION WITH SALES:
  loanamount vs sales: -0.0211
  loanamount vs leads: -0.0024
  loanamount vs qls: No correlation (constant values or insufficient data)
  loanamount vs l2s: -0.0211

CREDITSCORE:
----------------------------------------
Field type: Categorical

TOP 10 CATEGORIES BY SALES:
Category             Leads      QLS        Sales      L2S Rate  
----------------------------------------------------------------------
Excellent            2350       0          64         2.72      %
Good                 1098       0          33         3.01      %
Average              641        0          8          1.25      %
Below Average        507        0          1          0.20      %
Poor                 63         0          0          0.00      %
Unknown              5          0          0          0.00      %

Total categories: 6
Categories with sales > 0

In [128]:
# Create filtered dataset with only non-null enrichment values
print("="*60)
print("CREATING FILTERED DATASET FOR CORRELATION ANALYSIS")
print("="*60)

# Create analysis_df_inner with only records where ALL enrichment fields are not null
analysis_df_inner = analysis_df.dropna(subset=enrichment_fields)

print(f"Original dataset size: {len(analysis_df)}")
print(f"Filtered dataset size (all enrichment fields non-null): {len(analysis_df_inner)}")
print(f"Records removed: {len(analysis_df) - len(analysis_df_inner)}")
print(f"Percentage of data retained: {len(analysis_df_inner) / len(analysis_df) * 100:.2f}%")

# Show null rates in the filtered dataset
print(f"\nNull rates in filtered dataset:")
for field in enrichment_fields:
    null_count = analysis_df_inner[field].isnull().sum()
    print(f"  {field}: {null_count} nulls ({null_count / len(analysis_df_inner) * 100:.2f}%)")

print(f"\nFiltered dataset summary:")
print(f"  Total leads: {analysis_df_inner['leads'].sum()}")
print(f"  Total QLS: {analysis_df_inner['qls'].sum()}")
print(f"  Total sales: {analysis_df_inner['sales'].sum()}")
print(f"  Overall L2S rate: {(analysis_df_inner['sales'].sum() / analysis_df_inner['leads'].sum() * 100):.2f}%")


CREATING FILTERED DATASET FOR CORRELATION ANALYSIS
Original dataset size: 22008
Filtered dataset size (all enrichment fields non-null): 4663
Records removed: 17345
Percentage of data retained: 21.19%

Null rates in filtered dataset:
  loanamount: 0 nulls (0.00%)
  creditscore: 0 nulls (0.00%)

Filtered dataset summary:
  Total leads: 4664
  Total QLS: 0
  Total sales: 106
  Overall L2S rate: 2.27%


In [137]:
# CORRELATION ANALYSIS WITH FILTERED DATASET
print("="*60)
print("CORRELATION ANALYSIS WITH FILTERED DATASET")
print("="*60)

# Function to safely calculate correlation
def safe_correlation(series1, series2):
    try:
        # Remove null values
        data = pd.DataFrame({'x': series1, 'y': series2}).dropna()
        if len(data) < 2:
            return np.nan
        # Check if either series has zero variance
        if data['x'].std() == 0 or data['y'].std() == 0:
            return np.nan
        return data['x'].corr(data['y'])
    except:
        return np.nan

# Function to detect if field is numeric
def is_numeric_field(series):
    try:
        pd.to_numeric(series, errors='raise')
        return True
    except:
        return False

for field in enrichment_fields:
    print(f"\n{field.upper()}:")
    print("-" * 40)
    
    # Check if field is numeric
    is_numeric = is_numeric_field(analysis_df_inner[field])
    print(f"Field type: {'Numeric' if is_numeric else 'Categorical'}")
    print(f"Records with non-null {field}: {len(analysis_df_inner)}")
    
    if is_numeric:
        # Correlation analysis for numeric fields using filtered data
        print("\nCORRELATION WITH SALES (filtered dataset):")
        
        correlations = {
            'sales': safe_correlation(analysis_df_inner[field], analysis_df_inner['sales']),
            'leads': safe_correlation(analysis_df_inner[field], analysis_df_inner['leads']),
            'qls': safe_correlation(analysis_df_inner[field], analysis_df_inner['qls']),
            'l2s': safe_correlation(analysis_df_inner[field], analysis_df_inner['l2s'])
        }
        
        for metric, corr in correlations.items():
            if np.isnan(corr):
                print(f"  {field} vs {metric}: No correlation (constant values or insufficient data)")
            else:
                print(f"  {field} vs {metric}: {corr:.4f}")
                
        # Additional insights for numeric fields
        print(f"\nNumeric Field Insights:")
        print(f"  Min value: {analysis_df_inner[field].min()}")
        print(f"  Max value: {analysis_df_inner[field].max()}")
        print(f"  Mean value: {analysis_df_inner[field].mean():.2f}")
        print(f"  Median value: {analysis_df_inner[field].median():.2f}")
        print(f"  Std deviation: {analysis_df_inner[field].std():.2f}")
        
    else:
        # Categorical analysis using filtered data
        print("\nTOP 10 CATEGORIES BY SALES (filtered dataset):")
        cat_analysis = analysis_df_inner.groupby(field).agg({
            'leads': 'sum',
            'qls': 'sum',
            'sales': 'sum',
            'l2s': 'mean'
        }).round(4)
        
        # Calculate L2S rate safely (avoid division by zero)
        cat_analysis['l2s_rate'] = np.where(
            cat_analysis['leads'] > 0,
            (cat_analysis['sales'] / cat_analysis['leads'] * 100).round(2),
            0
        )
        
        # Sort by sales
        cat_analysis = cat_analysis.sort_values('sales', ascending=False)
        
        # Show top 10
        top_10 = cat_analysis.head(10)
        print(f"{'Category':<20} {'Leads':<10} {'QLS':<10} {'Sales':<10} {'L2S Rate':<10}")
        print("-" * 70)
        for category, row in top_10.iterrows():
            print(f"{str(category)[:18]:<20} {row['leads']:<10.0f} {row['qls']:<10.0f} {row['sales']:<10.0f} {row['l2s_rate']:<10.2f}%")
        
        print(f"\nTotal categories: {len(cat_analysis)}")
        print(f"Categories with sales > 0: {len(cat_analysis[cat_analysis['sales'] > 0])}")
        
        # Additional insights for categorical data
        print(f"\nCategory Performance Insights:")
        print(f"  Best L2S rate: {cat_analysis[cat_analysis['leads'] > 0]['l2s_rate'].max():.2f}%")
        print(f"  Worst L2S rate: {cat_analysis[cat_analysis['leads'] > 0]['l2s_rate'].min():.2f}%")
        print(f"  Average L2S rate: {cat_analysis[cat_analysis['leads'] > 0]['l2s_rate'].mean():.2f}%")
    
    print("=" * 60)


CORRELATION ANALYSIS WITH FILTERED DATASET

LOANAMOUNT:
----------------------------------------
Field type: Numeric
Records with non-null loanamount: 4663

CORRELATION WITH SALES (filtered dataset):
  loanamount vs sales: -0.0211
  loanamount vs leads: -0.0024
  loanamount vs qls: No correlation (constant values or insufficient data)
  loanamount vs l2s: -0.0211

Numeric Field Insights:
  Min value: 0.0
  Max value: 13233459.0
  Mean value: 337542.04
  Median value: 247500.00
  Std deviation: 583334.01

CREDITSCORE:
----------------------------------------
Field type: Categorical
Records with non-null creditscore: 4663

TOP 10 CATEGORIES BY SALES (filtered dataset):
Category             Leads      QLS        Sales      L2S Rate  
----------------------------------------------------------------------
Excellent            2350       0          64         2.72      %
Good                 1098       0          33         3.01      %
Average              641        0          8          1.

In [125]:
# INVESTIGATION: Leads vs Records Discrepancy
print("="*60)
print("INVESTIGATION: Leads vs Records Discrepancy")
print("="*60)

print(f"Total records: {len(analysis_df)}")
print(f"Total leads: {analysis_df['leads'].sum()}")
print(f"Average leads per record: {analysis_df['leads'].mean():.4f}")

print(f"\nLeads distribution:")
print(analysis_df['leads'].value_counts().sort_index().head(20))

print(f"\nRecords with leads > 1:")
leads_gt_1 = analysis_df[analysis_df['leads'] > 1]
print(f"Count: {len(leads_gt_1)}")
print(f"Percentage: {len(leads_gt_1) / len(analysis_df) * 100:.2f}%")

if len(leads_gt_1) > 0:
    print(f"\nTop records with highest leads:")
    print(leads_gt_1[['subid', 'leads', 'qls', 'sales']].sort_values('leads', ascending=False).head(10))

print(f"\nLeads statistics:")
print(f"Min: {analysis_df['leads'].min()}")
print(f"Max: {analysis_df['leads'].max()}")
print(f"Median: {analysis_df['leads'].median()}")
print(f"Std: {analysis_df['leads'].std():.2f}")

# Check if there are any records with 0 leads
zero_leads = analysis_df[analysis_df['leads'] == 0]
print(f"\nRecords with 0 leads: {len(zero_leads)}")
if len(zero_leads) > 0:
    print("This might explain the discrepancy - some records have 0 leads")


INVESTIGATION: Leads vs Records Discrepancy
Total records: 22008
Total leads: 22010
Average leads per record: 1.0001

Leads distribution:
leads
1    22006
2        2
Name: count, dtype: int64

Records with leads > 1:
Count: 2
Percentage: 0.01%

Top records with highest leads:
            subid  leads  qls  sales
16081  5dBAiZv3XM      2    0      0
17406  H6tVMjzhF7      2    0      0

Leads statistics:
Min: 1
Max: 2
Median: 1.0
Std: 0.01

Records with 0 leads: 0


In [ ]:
# 1. NULL RATES FOR ENRICHMENT FIELDS
print("="*60)
print("1. NULL RATES FOR ENRICHMENT FIELDS")
print("="*60)

# Identify enrichment fields (exclude subid, company, transaction_date, leads, qls, sales, l2s)
enrichment_fields = [col for col in analysis_df.columns if col not in ['subid', 'company', 'transaction_date', 'leads', 'qls', 'sales', 'l2s']]
print(f"Enrichment fields found: {enrichment_fields}")

# Fill NaN values in l2s with 0
analysis_df['l2s'] = analysis_df['l2s'].fillna(0)

null_rates = []
for field in enrichment_fields:
    total_rows = len(analysis_df)
    null_rows = analysis_df[field].isnull().sum()
    null_rate = (null_rows / total_rows) * 100
    
    null_rates.append({
        'field': field,
        'total_rows': total_rows,
        'null_rows': null_rows,
        'null_rate_pct': null_rate,
        'filled_rows': total_rows - null_rows
    })
    
    print(f"{field}:")
    print(f"  Total rows: {total_rows}")
    print(f"  Null rows: {null_rows}")
    print(f"  Filled rows: {total_rows - null_rows}")
    print(f"  Null rate: {null_rate:.2f}%")
    print("-" * 40)

# Create summary DataFrame
null_summary = pd.DataFrame(null_rates)
print("\nSUMMARY TABLE:")
print(null_summary[['field', 'null_rate_pct', 'filled_rows', 'total_rows']].round(2))
